# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [1]:
!pip -q install ./python

tensorflow 1.7.1 has requirement numpy>=1.13.3, but you'll have numpy 1.12.1 which is incompatible.
ipython 6.5.0 has requirement prompt-toolkit<2.0.0,>=1.0.15, but you'll have prompt-toolkit 3.0.5 which is incompatible.


The environments corresponding to both versions of the environment are already saved in the Workspace and can be accessed at the file paths provided below.  

Please select one of the two options below for loading the environment.

In [2]:
from unityagents import UnityEnvironment
import numpy as np

# select this option to load version 1 (with a single agent) of the environment
#env = UnityEnvironment(file_name='/data/Reacher_One_Linux_NoVis/Reacher_One_Linux_NoVis.x86_64')

# select this option to load version 2 (with 20 agents) of the environment
env = UnityEnvironment(file_name='/data/Reacher_Linux_NoVis/Reacher.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

Number of agents: 20
Size of each action: 4
There are 20 agents. Each observes a state with length: 33
The state for the first agent looks like: [  0.00000000e+00  -4.00000000e+00   0.00000000e+00   1.00000000e+00
  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00  -1.00000000e+01   0.00000000e+00
   1.00000000e+00  -0.00000000e+00  -0.00000000e+00  -4.37113883e-08
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   5.75471878e+00  -1.00000000e+00
   5.55726624e+00   0.00000000e+00   1.00000000e+00   0.00000000e+00
  -1.68164849e-01]


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agents while they are training**, and you should set `train_mode=True` to restart the environment.

In [5]:
env_info = env.reset(train_mode=True)[brain_name]      # reset the environment    
states = env_info.vector_observations                  # get the current state (for each agent)
scores = np.zeros(num_agents)                          # initialize the score (for each agent)
while True:
    actions = np.random.randn(num_agents, action_size) # select an action (for each agent)
    actions = np.clip(actions, -1, 1)                  # all actions between -1 and 1
    env_info = env.step(actions)[brain_name]           # send all actions to tne environment
    next_states = env_info.vector_observations         # get next state (for each agent)
    rewards = env_info.rewards                         # get reward (for each agent)
    dones = env_info.local_done                        # see if episode finished
    scores += env_info.rewards                         # update the score (for each agent)
    states = next_states                               # roll over states to next time step
    if np.any(dones):                                  # exit loop if episode finished
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.08449999811127781


When finished, you can close the environment.

In [6]:
#env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agents while they are training.  However, **_after training the agents_**, you can download the saved model weights to watch the agents on your own machine! 

In [7]:
import time 
import torch 

def ddpg(n_episodes=2000, max_t=1000):
    """Deep Deterministic Policy Gradients (DDPG).
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
    """
    scores = []      # episodic scores
    moving_avg = []  # moving average over 100 episodes and over all agents

    ## Perform n_episodes of training
    time_training_start = time.time()
    for i_episode in range(1, n_episodes+1):
        time_episode_start = time.time()
        agent.noise.reset()
        states = env.reset(train_mode=True)[brain_name].vector_observations
        scores_episode = np.zeros(num_agents)           

        for t in range(1, max_t+1):
            # Perform a step: S;A;R;S'
            actions = agent.act(states)                 
            env_info = env.step(actions)[brain_name]    
            rewards = env_info.rewards                  
            next_states = env_info.vector_observations 
            # Send the results to the Agent
            for (state, action, reward, next_state) \
                    in zip(states, actions, rewards, next_states):
                agent.step(state, action, reward, next_state)
            # Update the variables for the next iteration
            states = next_states
            scores_episode += rewards

        # Store the rewards and calculate the moving average
        scores.append(scores_episode.tolist())
        moving_avg.append(np.mean(scores[-100:], axis=0))
        # Calculate the elapsed time
        time_episode = time.time() - time_episode_start
        time_elapsed = time.time() - time_training_start
        time_episode_str = time.strftime('%Mm%Ss', time.gmtime(time_episode))


        print('Episode {:3d} ({})\tScore: {:5.2f} (H: {:5.2f} / L: {:5.2f})\t' \
              'Moving average: {:5.2f} (H: {:5.2f} / L: {:5.2f})' \
              .format(i_episode, time_episode_str, scores_episode.mean(),
                      scores_episode.max(), scores_episode.min(),
                      moving_avg[-1].mean(), moving_avg[-1].max(),
                      moving_avg[-1].min()))
        torch.save(agent.actor_local.state_dict(),'ddpg_actor.pth')
        torch.save(agent.critic_local.state_dict(),'ddpg_critic.pth')
        ## Check if the environment has been solved
        if moving_avg[-1].mean() >= 30 and i_episode >= 100:
            time_elapsed_str = time.strftime('%Hh%Mm%Ss', time.gmtime(time_elapsed))
            print('\nEnvironment solved in {:d} episodes!\t' \
                  'Average Score: {:.2f}\tElapsed time: {}'
                  .format(i_episode-100, moving_avg[-1].mean(),
                          time_elapsed_str))
            
            ## Save the model with parameters
            checkpoint = {
                'actor_dict': agent.actor_local.state_dict(),
                'critic_dict': agent.critic_local.state_dict(),
                'ddpg_params': {
                    'batch_size': ddpg_agent._batch_size,
                    'buffer_size': ddpg_agent._buffer_size,
                    'gamma': ddpg_agent._gamma,
                    'lr_actor': ddpg_agent._lr_actor,
                    'lr_critic': ddpg_agent._lr_critic,
                    'tau': ddpg_agent._tau,
                    },
                'episodes': i_episode,
                'elapsed_time': time_elapsed_str,
                'scores': scores,
                'moving_avg': moving_avg,
                }
            print('\nSaving model ...', end=' ')
            torch.save(checkpoint, 'checkpoint.pt')
            print('done.')

            break

    return scores, moving_avg

In [ ]:
# Create the Agent
from agent import Agent
agent = Agent(state_size, action_size, random_seed=0)

# Train the Agent
scores, moving_avg = ddpg()

DDPG Agent hyperparameters:
	batch size: 		     128
	buffer size: 		  100000
	discount / gamma: 	   0.990
	actor learning rate: 	   0.001
	critic learning rate: 	   0.001
	soft update / tau: 	   0.001
	noise decay rate: 	   0.999

Episode   1 (03m00s)	Score: 21.44 (H: 29.29 / L: 10.91)	Moving average: 21.44 (H: 29.29 / L: 10.91)
Episode   2 (03m03s)	Score: 28.56 (H: 38.35 / L: 10.87)	Moving average: 25.00 (H: 33.14 / L: 14.98)
Episode   3 (03m04s)	Score: 28.96 (H: 37.11 / L: 16.07)	Moving average: 26.32 (H: 32.88 / L: 18.82)
Episode   4 (03m03s)	Score: 33.93 (H: 38.87 / L: 29.42)	Moving average: 28.23 (H: 34.08 / L: 22.26)
Episode   5 (03m05s)	Score: 34.37 (H: 38.26 / L: 28.68)	Moving average: 29.46 (H: 34.04 / L: 24.60)
Episode   6 (03m07s)	Score: 36.45 (H: 39.54 / L: 27.72)	Moving average: 30.62 (H: 34.75 / L: 26.05)
Episode   7 (03m07s)	Score: 33.42 (H: 38.61 / L: 19.10)	Moving average: 31.02 (H: 34.62 / L: 26.70)
Episode   8 (03m07s)	Score: 30.91 (H: 38.60 / L:  0.77)	Moving averag